In [1]:
import os
import cv2
import numpy as np

In [2]:
open_pic_path = next(os.walk("img/OpenFace"))[2]
open_pic_path = [x for x in open_pic_path if ".jpg" in x]
len(open_pic_path)

1231

In [3]:
close_pic_path = next(os.walk("img/ClosedFace"))[2]
close_pic_path = [x for x in close_pic_path if ".jpg" in x]
len(close_pic_path)

1192

In [4]:
train_open_pic_path = open_pic_path[:1181]
train_close_pic_path = close_pic_path[:1142]
test_open_pic_path = open_pic_path[1181:]
test_close_pic_path = close_pic_path[1142:]

In [5]:
train_picture_path = train_open_pic_path+ train_close_pic_path
test_picture_path = test_open_pic_path + test_close_pic_path

In [6]:
from tensorflow.keras.models import load_model
import torchvision.transforms.functional as TF
import torch

In [7]:
def preprocess_image(image):
    image = TF.to_pil_image(image)
    image = TF.resize(image, (128, 128))
    image = TF.to_tensor(image)
    image = (image - image.min())/(image.max() - image.min())
    image = (2 * image) - 1
    image = torch.transpose(image,0,2).detach().numpy()
    return image

## Open = 1
## Closed = 0

In [8]:
def generate_combine_data(pp_):
    X = []
    Y = []
    for path in pp_:
        if "closed" in path:
            img = cv2.imread(f"img/ClosedFace/{path}")
            Y.append(0)
        else:
            img = cv2.imread(f"img/OpenFace/{path}")    
            Y.append(1)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        preprocessed_image = preprocess_image(img)
        X.append(preprocessed_image)
    return np.array(X),np.array(Y)

In [9]:
train_inputs,train_labels = generate_combine_data(train_picture_path)
test_inputs,test_labels = generate_combine_data(test_picture_path)

In [10]:
from tensorflow.keras.layers import Flatten,Dense,Conv2D,MaxPooling2D,Activation
from tensorflow.keras import Sequential
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [11]:
model = Sequential([
        Conv2D(128,3,padding="same",input_shape=(128,128,1)),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2),padding="same"),
        Conv2D(64,3,padding="same"),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2),padding="same"),
        Conv2D(32,3,padding="same"),
        Activation('relu'),
        Conv2D(16,3,padding="same"),
        Activation('relu'),
        Conv2D(8,3,padding="same"),
        Activation('relu'),
        Flatten(),
        Dense(2)
])

In [12]:
model.compile(
              optimizer = 'adam',
              loss = SparseCategoricalCrossentropy(from_logits=True),
              metrics = ['accuracy'],
             )

In [13]:
model.fit(train_inputs, train_labels, epochs=25,  validation_split = 0.1)

Epoch 1/25
66/66 [==============================] - 5s 32ms/step - loss: 0.6343 - accuracy: 0.6423 - val_loss: 0.7632 - val_accuracy: 0.6094
Epoch 2/25
66/66 [==============================] - 1s 22ms/step - loss: 0.4031 - accuracy: 0.8236 - val_loss: 0.3173 - val_accuracy: 0.8498
Epoch 3/25
66/66 [==============================] - 1s 21ms/step - loss: 0.3526 - accuracy: 0.8590 - val_loss: 0.2894 - val_accuracy: 0.8584
Epoch 4/25
66/66 [==============================] - 1s 21ms/step - loss: 0.2389 - accuracy: 0.9112 - val_loss: 0.8298 - val_accuracy: 0.6781
Epoch 5/25
66/66 [==============================] - 1s 21ms/step - loss: 0.1830 - accuracy: 0.9262 - val_loss: 0.3410 - val_accuracy: 0.8670
Epoch 6/25
66/66 [==============================] - 1s 21ms/step - loss: 0.1100 - accuracy: 0.9596 - val_loss: 0.2412 - val_accuracy: 0.9013
Epoch 7/25
66/66 [==============================] - 1s 21ms/step - loss: 0.0825 - accuracy: 0.9673 - val_loss: 0.4558 - val_accuracy: 0.8712
Epoch 8/25
66

In [14]:
model.save("open_close_baseline.h5")

In [15]:
def get_confusion(inputs,labels):
    TP = 0
    FN = 0
    TN = 0
    FP = 0
    for inp,label in zip(inputs,labels):
        predict = np.array(model.predict(np.expand_dims(inp,0))[0])
        predict = np.argmax(predict)
        if label ==1 and predict == label:
            TP += 1
        if label ==1 and predict != label:
            FN += 1
        if label ==0 and predict == label:
            TN += 1
        if label ==0 and predict != label:
            FP += 1
    ACC = (TP+TN) / (TP+FP+FN+TN)
    PRECISION = TP / (TP+FP)
    RECALL = TP / (TP+FN)
    F1_SCORE = 2/((1/PRECISION) + (1/RECALL))
    return ACC,PRECISION,RECALL,F1_SCORE

In [16]:
ACC,PRECISION,RECALL,F1_SCORE = get_confusion(train_inputs,train_labels)

In [17]:
print(f"Train ACC --- {ACC}")
print(f"Train PRECISION --- {PRECISION}")
print(f"Train RECALL --- {RECALL}")
print(f"Train F1_SCORE --- {F1_SCORE}")

Train ACC --- 0.9875161429186397
Train PRECISION --- 0.9760330578512396
Train RECALL --- 1.0
Train F1_SCORE --- 0.9878711836051862


In [18]:
ACC,PRECISION,RECALL,F1_SCORE = get_confusion(test_inputs,test_labels)

In [19]:
print(f"Test ACC --- {ACC}")
print(f"Test PRECISION --- {PRECISION}")
print(f"Test RECALL --- {RECALL}")
print(f"Test F1_SCORE --- {F1_SCORE}")

Test ACC --- 0.93
Test PRECISION --- 0.9574468085106383
Test RECALL --- 0.9
Test F1_SCORE --- 0.9278350515463918
